In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn


In [1]:
import os
import csv
import mediapipe as mp
import cv2

2024-12-21 06:34:13.325539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-21 06:34:13.347621: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 06:34:13.347660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 06:34:13.360991: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [4]:
# 의미 있는 랜드마크만 선택 (Left/Right shoulders, elbows, wrists, pinkies, indices, thumbs)
selected_landmarks = [
    mp_pose.PoseLandmark.LEFT_SHOULDER,
    mp_pose.PoseLandmark.RIGHT_SHOULDER,
    mp_pose.PoseLandmark.LEFT_ELBOW,
    mp_pose.PoseLandmark.RIGHT_ELBOW,
    mp_pose.PoseLandmark.LEFT_WRIST,
    mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.LEFT_PINKY,
    mp_pose.PoseLandmark.RIGHT_PINKY,
    mp_pose.PoseLandmark.LEFT_INDEX,
    mp_pose.PoseLandmark.RIGHT_INDEX,
    mp_pose.PoseLandmark.LEFT_THUMB,
    mp_pose.PoseLandmark.RIGHT_THUMB,
]

In [5]:
# CSV 파일 컬럼 초기화
landmarks = ['class', 'posture_type', 'person_id']
for landmark in selected_landmarks:
    landmarks += [f'{landmark.name}_x', f'{landmark.name}_y', f'{landmark.name}_z']

In [6]:
# CSV 파일 초기화
output_csv = 'lateralraise_coords_personid_filtered.csv'
with open(output_csv, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
# 데이터 저장 함수 수정
def export_landmark(results, action_label, posture_type, person_id):
    try:
        # 필요한 랜드마크만 추출 (x, y, z 좌표)
        keypoints = [action_label, posture_type, person_id]
        for landmark_id in selected_landmarks:
            landmark = results.pose_landmarks.landmark[landmark_id]
            keypoints.extend([landmark.x, landmark.y, landmark.z])  # visibility 제외
        
        # 데이터 저장
        with open(output_csv, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(f"Error: {e}")

In [8]:
# 이미지 경로 설정
base_path = r"/root/juno/Day16_201016_F_SLR"
folders = [os.path.join(base_path, folder) for folder in os.listdir(base_path)]

In [9]:
# Mediapipe Pose로 데이터 추출 및 라벨링
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for folder in folders:
        for subfolder, _, files in os.walk(folder):
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일 필터링
                    file_path = os.path.join(subfolder, file)
                    
                    # Action Label: 파일명 앞 3자리 숫자 추출
                    action_label = file.split('-')[0]
                    
                    # Posture Type 결정
                    posture_type = 'correct' if action_label == '377' else 'incorrect'
                    
                    # Person ID 추출: 'Z'로 시작하는 부분 추출
                    person_id = next((part for part in file.split('-') if part.startswith('Z')), 'Unknown')
                    
                    # 이미지 읽기
                    image = cv2.imread(file_path)
                    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    # Mediapipe로 Pose 추정
                    results = pose.process(image_rgb)
                    
                    # Landmarks 추출 및 CSV 저장
                    if results.pose_landmarks:
                        export_landmark(results, action_label, posture_type, person_id)
                        print(f"Processed: {file} | Label: {action_label} | Type: {posture_type} | ID: {person_id}")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1734688724.280674   11453 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734688724.303962   11453 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1734688724.337954   11481 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Processed: 377-2-1-18-Z52_A-0000001.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000002.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000003.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000004.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000005.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000006.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000007.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000008.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000009.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000010.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000011.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-0000012.jpg | Label: 377 | Type: correct | ID: Z52_A
Processed: 377-2-1-18-Z52_A-